$$
%\newcommand{\oneform}[1]{{\vphantom{{#1}}}^{1}\!{#1}{\vphantom{#1}}}
\newcommand{\oneform}[1]{\overset{1}{#1}{\vphantom{#1}}}
%\newcommand{\volform}[1]{{\vphantom{{\omega}}}^{#1}\!{\omega}{\vphantom{\omega}}}
\newcommand{\volform}[1]{\overset{#1}{\omega}{\vphantom{\omega}}}
\renewcommand{\vector}[1]{\boldsymbol{#1}}
\newcommand{\curve}[1]{{#1}}
\newcommand{\fbasis}[1]{{d#1}}
\newcommand{\uprm}[1]{^{\mathrm{#1}}}
\newcommand{\tensor}[1]{\mathbf{#1}}
\newcommand{\norm}[1]{||#1||}
$$

# Overview

In this exercise we investigate a capacitor-like structure surrounded by nothing but air as depicted in the section on the geometry below.
The role of the dielectric medium is played by a insulating elastomer. The electrodes are modeled by an insulating but highly permittive (very high $\varepsilon$) elastic material. Both the elastomer and the electrodes are assumed to be pratically *rigid*. The simplified electrostatic model of the electrodes
will be assessed a posteriori. 

We employ the vacuum co-energy density
\begin{equation}
\psi^{\text{vac}}(\norm{\oneform{e}}) = -\frac{\varepsilon_0}{2} \norm{\oneform{e}}^2
\end{equation}
for the air. For dielectric media (electrode and elastomer), we employ
\begin{equation}
\psi^{\text{ES}}(\norm{\oneform{e}}) = -\frac{\varepsilon_0\,\epsilon_r}{2} \norm{\oneform{e}}^2 ,
\end{equation}
which is a rather simple model but sufficient for our purposes.

# Geometry

<img src=capacitor.png></img>

In [328]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

In [329]:
import numpy as np

In [330]:
L = 10
l = 1
d = 1/10
r = d/4

In [331]:
everywhere = Rectangle(L, L).Face()

In [332]:
elastomer = Rectangle(l, l).Face()
inclusion_1 = Circle((0.2,0.2),0.1).Face()
inclusion_2 = Circle((0.6,0.6),0.1).Face()
inclusion_3 = Circle((0.8,0.8),0.1).Face()

inclusions = inclusion_1 + inclusion_2 + inclusion_3

elastomer = elastomer - inclusions


In [333]:
electrode = WorkPlane()
electrode.MoveTo(0, l).Line(l)
electrode.Direction(0, 1).Line(d-r)
electrode.Arc(r, 90)
electrode.Direction(-1, 0).Line(l-r)
electrode.Close()
electrode = electrode.Face()

In [334]:
elastomer.maxh = l/10
electrode.maxh = d/2

for f in elastomer.faces:
    f.name = "elastomer"
    
for f in electrode.faces:
    f.name = "electrode"

for f in inclusions.faces:
    f.name = "inclusion"

bodies = Glue([elastomer, electrode, inclusions])

In [335]:

air = everywhere - bodies

    
for f in air.faces:
    f.name = "air"

all_space = Glue([bodies, air])

In [336]:
for e in all_space.edges[X < 0 + 1e-4]:
    e.name = "left"
    
for e in all_space.edges[X > L - 1e-4]:
    e.name = "right"
    
for e in all_space.edges[Y < 0 + 1e-4]:
    e.name = "bottom"
    
for e in all_space.edges[Y > L - 1e-4]:
    e.name = "top"

In [337]:
DrawGeo(all_space)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [5.0, 5.000000000000001, 0.…

BaseWebGuiScene

In [338]:
geo = OCCGeometry(all_space, dim=2)
ngmesh = geo.GenerateMesh(maxh=0.5)

In [339]:
from ngsolve import *
from ngsolve.webgui import Draw

In [340]:
mesh = Mesh(ngmesh)

In [341]:
Draw(mesh)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 1, 'order3d':…

BaseWebGuiScene

In [342]:
AIR = mesh.Materials("air")
ELASTOMER = mesh.Materials("elastomer")
ELECTRODE = mesh.Materials("electrode")
INCLUSIONS = mesh.Materials("inclusion")

# Function spaces

In [343]:
Integrate(CF(1), mesh, definedon=ELASTOMER)
Integrate(CF(1), mesh, definedon=INCLUSIONS)

0.08997955033585597

In [344]:
fes = H1(mesh, order=2, dirichlet="bottom") * H1(mesh, order=2, dirichlet="top|bottom")

phi, phi_m = fes.TrialFunction()

gfsol = GridFunction(fes)
gf_phi, gf_phi_m = gfsol.components

In [345]:
# for postprocessing
fes_e = VectorL2(mesh, order=1) # for electric field and d-field

In [346]:
gf_e = GridFunction(fes_e)
gf_d = GridFunction(fes_e)
gf_h = GridFunction(fes_e)
gf_b = GridFunction(fes_e)

# Kinematics

In [347]:
# The electric field
def e(phi):
    return -Grad(phi)

# The h-field

def h(phi_m):
    return -Grad(phi_m)

# Energy densities

## Electrostatic energy density

In [348]:
epsilon_0 = 8.856*1e-7
epsilon_r_air = Parameter(1)
epsilon_r_electrode = Parameter(1e4)
epsilon_r_elastomer = Parameter(5)
epsilon_r_inclusion = Parameter(50)
mu_0 = Parameter(4*np.pi*1e-1)
mu_r_inclusion = Parameter(10)
mu_r_nm = Parameter(1)

def Psi(e, h, epsilon_r, mu_r):
    return -epsilon_0 * epsilon_r * InnerProduct(e, e) -mu_0  * mu_r * InnerProduct(h, h) 

In [349]:
def generate_Psi_dict(e, h):
    return {AIR: Psi(e, h, epsilon_r_air, mu_r_nm), 
            ELASTOMER: Psi(e, h, epsilon_r_elastomer, mu_r_nm ),
            ELECTRODE: Psi(e, h, epsilon_r_electrode, mu_r_nm),
            INCLUSIONS: Psi(e, h, epsilon_r_inclusion, mu_r_inclusion)}

## Postprocessing helper for the electric displacement field (aka charge potential)

In [350]:
def db_dict(e, h, Psi_dict=None):
    e.MakeVariable()
    h.MakeVariable()
    Psi_dict = generate_Psi_dict(e, h) if Psi_dict is None else Psi_dict
    return {domain: {"d": -Psi.Diff(e), "b": -Psi.Diff(h)} for domain, Psi in Psi_dict.items()}

# Loads

The only loading comes from a prescribed charge density in the electrode. For modling purposes we may assume
the charge is equally distributed. In reality, this is of course not the case. In particular, for conductors
one would expect that charges gather near the surface, not in the bulk. We will assess the validity of this 
modeling assumption a posteriori.

In [351]:
q = Parameter(0.0)

h_infty = Parameter(0.0)
gf_bc = GridFunction(fes)

# Govering potential

In [352]:
Pi = BilinearForm(fes, symmetric=True)
Pi += Variation(
    sum([Psi * dx(domain) for domain, Psi in generate_Psi_dict(e(phi), h(phi_m)).items()])
    + (-phi * q * dx(ELECTRODE))
)

In [353]:
# Create the vector holding the discrete variation
rhs = gfsol.vec.CreateVector()

# Compute the variation; evaluate with the data of gfu
Pi.Apply(gfsol.vec, rhs)

In [354]:
Norm(rhs)

0.0

## Posprocessing equations

In [356]:
pp_db_dict = db_dict(e(gf_phi), h(gf_phi_m))

In [322]:
vtk = VTKOutput(mesh, coefs=[gf_e, gf_d, gfphi], names=['gf_e', 'gf_d', 'gfphi'], filename = '/home/tellocam/CSE/NLCFP/Ex4/vtk_outputs/output')

def pp():
    gf_e.Interpolate(e(gfphi))
    for domain, expr in pp_d_dict.items():
        gf_d.Interpolate(expr, definedon=domain)
    vtk.Do()

# Run the problem

In [323]:
from ngsolve.solvers import Newton

In [324]:
gfphi.vec[:] = 0
scene = Draw(gfphi)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

In [325]:
gfphi.vec[:] = 0
for val in np.linspace(0, 5e-3, 11):
    q.Set(val)
    gf_bc.components[1].Set(h_infty * L, mesh.Boundaries("top"))
    success, niter = Newton(Pi, gfphi, inverse="pardiso", dirichletvalues=gf_bc.vec)
    if success != 0:
        raise Exception("Newton did not converge")
        
    pp()
    
    print("\n" + "load parameter val = {0!s}".format(val) + "\n")
    scene.Redraw()

Newton iteration  0
err =  0.0

load parameter val = 0.0

Newton iteration  0
err =  0.0141325292877771
Newton iteration  1
err =  7.502654329580973e-14

load parameter val = 0.0005

Newton iteration  0
err =  0.014132529287777095
Newton iteration  1
err =  8.606919550616275e-14

load parameter val = 0.001

Newton iteration  0
err =  0.0141325292877771
Newton iteration  1
err =  7.270451840811402e-14

load parameter val = 0.0015

Newton iteration  0
err =  0.014132529287777099
Newton iteration  1
err =  7.800825877963889e-14

load parameter val = 0.002

Newton iteration  0
err =  0.014132529287777097
Newton iteration  1
err =  8.179044617421094e-14

load parameter val = 0.0025

Newton iteration  0
err =  0.014132529287777099
Newton iteration  1
err =  7.193042102840026e-14

load parameter val = 0.003

Newton iteration  0
err =  0.01413252928777709
Newton iteration  1
err =  7.039542681726174e-14

load parameter val = 0.0035

Newton iteration  0
err =  0.014132529287777102
Newton iterat

In [326]:
pp()
Draw(gfphi)
Draw(gf_e)
Draw(gf_d)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

# TODO

 * Write phi, e and d to vtk files: The result should be one file per load step holding data for all three fields plus a "master" file (.pvd) to be opened with Paraview that lets the user walk through time steps.
 * Add dielectric circular inclusions in the elastomer as another material.
 * Add magnetostatics: the circular inclusions are linearly magnetic, everything else is nonmagnetic. The whole system is exposed to an external uniform magnetic field. Which boundary conditions represent the best approximation of this situation? We use $\phi$